In [ ]:
import SimpleITK as sitk
import numpy as np
from skimage import filters, util
from skimage.segmentation import slic
import os
import pandas as pd
from scipy.stats import skew, kurtosis
from glob import glob
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from mirp import extract_features



# Load iamge
def load_nifti_image(nifti_path):
    image = sitk.ReadImage(nifti_path)
    image = sitk.Cast(image, sitk.sitkFloat32)
    return image

# resampling
def resample_image(image, new_spacing=(1, 1, 1), interpolator=sitk.sitkBSpline):
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    new_size = [
        int(round(original_size[0] * (original_spacing[0] / new_spacing[0]))),
        int(round(original_size[1] * (original_spacing[1] / new_spacing[1]))),
        int(round(original_size[2] * (original_spacing[2] / new_spacing[2]))),
    ]
    
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetInterpolator(interpolator)
    resampled_image = resample.Execute(image)
    return resampled_image

# normalization
def normalize_image(image):
    array = sitk.GetArrayFromImage(image).astype(np.float32)
    array = (array - np.min(array)) / (np.max(array) - np.min(array))
    normalized_image = sitk.GetImageFromArray(array)
    normalized_image.CopyInformation(image)
    return normalized_image

# image preprocessing
def process_mri_image(input_nii_path, output_nii_path):
    image = load_nifti_image(input_nii_path)
    resampled_image = resample_image(image, new_spacing=[1.0, 1.0, 1.0])
    normalized_image = normalize_image(resampled_image)
    sitk.WriteImage(normalized_image, output_nii_path)

# example usage
input_path = ".../ID.nrrd"  #your input file path
output_path = ".../ID.nii.gz"  

process_mri_image(input_path, output_path)



# local entropy map calculation
def process_image(image_path, output_path):
    
    # load image
    image = sitk.ReadImage(image_path)
    image_array = sitk.GetArrayFromImage(image).astype(np.float32)

    # normalization
    image_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array))

    # calculating local entropy map
    neighborhood_size = 3
    footprint = np.ones((neighborhood_size, neighborhood_size, neighborhood_size), dtype=np.uint8)
    image_array_uint8 = util.img_as_ubyte(image_array)
    entropy_image = filters.rank.entropy(image_array_uint8, footprint)

    # save local entropy map
    entropy_image_sitk = sitk.GetImageFromArray(entropy_image)
    entropy_image_sitk.CopyInformation(image)
    sitk.WriteImage(entropy_image_sitk, output_path)

# example usage
input_files = ".../ID.nii.gz"
output_directory = ".../ID_entropy.nii.gz"

process_image(input_files,output_directory)



# supervoxel segmentation using MRI and entropy map
# normalization (masked region)
def min_max_normalization(image_array, mask_array):
    masked_data = image_array[mask_array > 0]
    min_val = np.min(masked_data)
    max_val = np.max(masked_data)
    normalized_array = (image_array - min_val) / (max_val - min_val)
    return normalized_array

# determining the number of supervoxels according to tumor volume
def determine_superpixels(mask_array):
    mask_sum = np.sum(mask_array> 0) 
    if mask_sum < 1000:
        return 30
    elif 1000 <= mask_sum < 10000:
        return 50
    elif 10000 <= mask_sum < 100000:
        return 60
    else:
        return 80

# SLIC
def process_mri_and_entropy(mri_image_path, entropy_image_path, tumor_mask_path, output_dir):
        
        # Load MRI, entropy, and tumor mask images
        mri_image = sitk.ReadImage(mri_image_path)
        entropy_image = sitk.ReadImage(entropy_image_path)
        tumor_mask = sitk.ReadImage(tumor_mask_path)

        # Convert images to numpy arrays and downcast to float32
        mri_array = sitk.GetArrayFromImage(mri_image).astype(np.float32)
        entropy_array = sitk.GetArrayFromImage(entropy_image).astype(np.float32)
        tumor_mask_array = sitk.GetArrayFromImage(tumor_mask).astype(np.float32)

        # normalization (masked region)
        mri_array_normalized = min_max_normalization(mri_array, tumor_mask_array)
        entropy_array_normalized = min_max_normalization(entropy_array, tumor_mask_array)

        # number of supervoxels
        num_superpixels = determine_superpixels(tumor_mask_array)

        # Combine MRI and entropy values into a multi-channel image
        multi_channel_image = np.stack([mri_array_normalized, entropy_array_normalized], axis=-1)

        # Apply 3D SLIC segmentation to the masked region
        segmented_tumor_region = slic(multi_channel_image, n_segments=num_superpixels, compactness=0.05, mask=tumor_mask_array)

        # save
        segmented_tumor_image = sitk.GetImageFromArray(segmented_tumor_region)
        segmented_tumor_image.CopyInformation(mri_image)
        
        # Save the segmented tumor region
        sitk.WriteImage(segmented_tumor_image, output_dir)

# example usage
mri_image_dir = '.../ID.nii.gz'
entropy_image_dir = '.../ID_entropy.nii.gz'
tumor_mask_dir = '.../Segmentation.seg.nrrd'
output_dir = '.../supervoxel.nii.gz'

process_mri_and_entropy(mri_image_dir, entropy_image_dir, tumor_mask_dir, output_dir)



# feature extraction from supervoxels
# Define the extracted features
def extract_statistical_features(data):
    features = [
        np.mean(data),
        np.median(data),
        np.quantile(data, 0.25),
        np.quantile(data, 0.75),
        np.quantile(data, 0.75) - np.quantile(data, 0.25),  # IQR
        np.std(data),
        np.var(data),
        skew(data),
        kurtosis(data),
        np.sum(data**2) / len(data)  # 能量
    ]
    return features

# extract features from MRI and entropy map
def process_superpixels(mri_image_path, entropy_image_path, superpixel_mask_path, output_dir):
        # load image
        mri_image = sitk.ReadImage(mri_image_path)
        entropy_image = sitk.ReadImage(entropy_image_path)
        superpixel_mask = sitk.ReadImage(superpixel_mask_path)

        # Convert images to numpy arrays and downcast to float32
        mri_array = sitk.GetArrayFromImage(mri_image).astype(np.float32)
        entropy_array = sitk.GetArrayFromImage(entropy_image).astype(np.float32)
        superpixel_mask_array = sitk.GetArrayFromImage(superpixel_mask).astype(np.int32)

        # normalization (masked region)
        mri_array_normalized = min_max_normalization(mri_array, superpixel_mask_array)
        entropy_array_normalized = min_max_normalization(entropy_array, superpixel_mask_array)

        # Initialize a list to store the features of each supervoxel
        features_list = []
        labels_list = []

        # Extracting superpixel labels
        unique_labels = np.unique(superpixel_mask_array)
        for label in unique_labels:
            if label == 0:  # Skip Background
                continue
            mask = (superpixel_mask_array == label)
            mri_data = mri_array_normalized[mask]
            entropy_data = entropy_array_normalized[mask]

            # feature extraction
            mri_features = extract_statistical_features(mri_data)
            entropy_features = extract_statistical_features(entropy_data)
            
            # Combining all features and labels
            combined_features = mri_features + entropy_features
            features_list.append(combined_features)
            labels_list.append(label)

        # Define feature name
        columns = ['MRI_mean', 'MRI_median', 'MRI_q1', 'MRI_q3', 'MRI_iqr', 'MRI_std', 'MRI_variance', 'MRI_skewness', 'MRI_kurtosis', 'MRI_energy',
                   'Entropy_mean', 'Entropy_median', 'Entropy_q1', 'Entropy_q3', 'Entropy_iqr', 'Entropy_std', 'Entropy_variance', 'Entropy_skewness', 'Entropy_kurtosis', 'Entropy_energy']

        # sava file
        features_df = pd.DataFrame(features_list, index=labels_list, columns=columns)
        sample_number = os.path.basename(mri_image_path)
        features_file_path = os.path.join(output_dir, f'{sample_number}.csv')
        features_df.to_csv(features_file_path, index_label='Superpixel_Label')

# example usage
mri_image_dir = '.../ID.nii.gz'
entropy_image_dir = '.../ID_entropy.nii.gz'
superpixel_mask_dir = '.../supervoxel.nii.gz'
output_dir = '.../supervoxel_features'

process_superpixels(mri_image_dir, entropy_image_dir, superpixel_mask_dir, output_dir)




# subregion clustering based on supervoxel features
# load feature file
def load_features(file):
    df = pd.read_csv(file)
    return df

# feature standardization
def preprocess_features(features_df):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_df.iloc[:, 1:])  # 排除超像素索引
    return features_scaled

# clustering
def k_clustering(features_scaled, n_clusters):
    clustering = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = clustering.fit_predict(features_scaled)
    return cluster_labels

# Map the clustering results back to each supervoxel label
def map_clusters_to_mask(cluster_labels, features_df, mask_file, output_dir):
    mask = sitk.ReadImage(mask_file)
    mask_data = sitk.GetArrayFromImage(mask)

    # Create a new mask to store the clustering results
    new_mask_data = np.zeros_like(mask_data, dtype=np.int32)

    # keep the background at 0
    cluster_labels = cluster_labels + 1
    
    # Replace the supervoxels lable with the corresponding cluster label
    for superpixel_id, label in zip(features_df.iloc[:, 0], cluster_labels):
        new_mask_data[mask_data == superpixel_id] = label
    
    # save multiregion masks
    new_mask = sitk.GetImageFromArray(new_mask_data)
    new_mask.CopyInformation(mask) 
    output_file = os.path.join(output_dir, f'subregion_{os.path.basename(mask_file)}')
    sitk.WriteImage(new_mask, output_file)

# example usage
features_dir = '.../supervoxel_features/ID.nii.gz.csv'  
mask_dir = '.../supervoxel.nii.gz' 
output_dir = '...' 

features_df = load_features(features_dir)    
features_scaled = preprocess_features(features_df)
n_clusters = 2  # the number of subregions used in this study
cluster_labels = k_clustering(features_scaled, n_clusters)
map_clusters_to_mask(cluster_labels, features_df, mask_dir, output_dir)



# subregion mask extraction
# load image
multiregion_mask_path = ".../clustered_supervoxel.nii.gz"
original_mask = sitk.ReadImage(multiregion_mask_path)
original_array = sitk.GetArrayFromImage(original_mask)

# subregion mask extraction
mask_1 = (original_array == 1).astype(np.uint8)  
mask_2 = (original_array == 2).astype(np.uint8)

# save subregion mask
def create_submask(array, reference_image):
    submask = sitk.GetImageFromArray(array)
    submask.CopyInformation(reference_image) 
    return submask

submask_1 = create_submask(mask_1, original_mask)
submask_2 = create_submask(mask_2, original_mask)

# output
output_dir = " "
sitk.WriteImage(submask_1, output_dir + "submask_1.nii.gz")
sitk.WriteImage(submask_2, output_dir + "submask_2.nii.gz")




# feature extraction from subregion masks
def feature_extract(img, mask):
    feature_data = extract_features(
        image_modality="mr",
        image=img,
        mask=mask,
        byslice=False,
        anti_aliasing=False, # to match the ISBI reference value
        intensity_normalisation="standardisation",
        base_discretisation_method="fixed_bin_number",
        base_discretisation_n_bins=32,
        glcm_spatial_method="3d_average",
        glrlm_spatial_method="3d_average"
    )
    return feature_data

# example usage (subregion 1)
mri_image_dir = ""
submask_dir = ".../submask_1.nii.gz"
output_file = ".../subregion_1.csv"

# Extract features 
df = pd.DataFrame()
data = feature_extract(mri_image_dir, submask_dir)
df_add = pd.DataFrame(data[0])
df = pd.concat([df, df_add])
df.to_csv(output_file, index=False)
